In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import pandas as pd

file_path = '/kaggle/input/wafer-defect-datasets/LSWMD.pkl'

# Load dataset
try:
    df = pd.read_pickle(file_path)
    print(" Dataset loaded successfully!")
    display(df.head())
except Exception as e:
    print(" Error loading file:", e)


In [ ]:
df.head(50)

In [ ]:
df.tail(50)


In [ ]:
df.columns
df.shape
df.dtypes

In [ ]:
print(df['failureType'].tail(1000).value_counts(dropna=False))
print(df['trianTestLabel'].tail(1000).value_counts(dropna=False))

In [ ]:
print("811455 failureType:", df.at[811455, 'failureType'], "Type:", type(df.at[811455, 'failureType']))
print("811455 trianTestLabel:", df.at[811455, 'trianTestLabel'], "Type:", type(df.at[811455, 'trianTestLabel']))


In [ ]:
import numpy as np


df['failureType'] = df['failureType'].apply(lambda x: [['del']] if isinstance(x, np.ndarray) and x.size == 0 else x)
df['trianTestLabel'] = df['trianTestLabel'].apply(lambda x: [['del']] if isinstance(x, np.ndarray) and x.size == 0 else x)


print(df.loc[811455:811456, ['failureType', 'trianTestLabel']])

In [ ]:
import numpy as np


def replace_empty(val):
    if isinstance(val, (list, np.ndarray)) and len(val) == 0:
        return [["del"]]
    return val

# Step 2: Replace [["none"]] with [["normal"]]
def replace_none(val):
    if isinstance(val, (list, np.ndarray)) and np.array_equal(val, [["none"]]):
        return [["normal"]]
    return val

# Apply to relevant columns only
df["failureType"] = df["failureType"].apply(replace_empty).apply(replace_none)
df["trianTestLabel"] = df["trianTestLabel"].apply(replace_empty)


In [ ]:
# Preview replaced rows
print(df[(df["failureType"].apply(lambda x: x == [["del"]]) | df["failureType"].apply(lambda x: x == [["normal"]]) |
          df["trianTestLabel"].apply(lambda x: x == [["del"]]))].tail(15))

In [ ]:
print(df['failureType'].tail(1000).value_counts(dropna=False))
print(df['trianTestLabel'].tail(1000).value_counts(dropna=False))

In [ ]:
df_clean = df[~(
    df['failureType'].apply(lambda x: x == [['del']]) |df['trianTestLabel'].apply(lambda x: x == [['del']])
)].reset_index(drop=True)


In [ ]:
print(df[['failureType', 'trianTestLabel']].tail(50))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the start and end of the index range
start_index = 811443
end_index = 811454

# Loop through the specified index range
for i in range(start_index, end_index + 1):
    if i in df.index:  # Make sure the index exists in the DataFrame
        wafer = df.loc[i]
        img_array = np.array(wafer['waferMap'])     # Convert list to NumPy array
        label = wafer['failureType']                # Get cleaned label

        plt.imshow(img_array, cmap='gray')          # Show as grayscale image
        plt.title(f"Index: {i}, Label: {label}")
        plt.axis('off')
        plt.show()
    else:
        print(f"Index {i} not found in DataFrame.")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Folder to store images
base_dir = "/content/wafer_images_last_1000"
os.makedirs(base_dir, exist_ok=True)

# Save images
for i in tqdm(range(max(0, len(df) - 1000), len(df))): # Iterate through the last 1000 rows of df
    row = df.iloc[i]
    image = np.array(row['waferMap'])
    label = row['failureType']

    # Create folder if it doesn't exist
    # Convert the label to a string for the folder name
    label_folder = os.path.join(base_dir, str(label))
    os.makedirs(label_folder, exist_ok=True)

    # File path
    filename = f"wafer_{i}.png"
    file_path = os.path.join(label_folder, filename)

    # Save image
    plt.imsave(file_path, image, cmap='gray')

In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# Paths
source_dir = "/content/wafer_images"         # Where all images are now
train_dir = "/content/wafer_split/train"     # 80%
test_dir = "/content/wafer_split/test"      # 20%

# Create target dirs
for path in [train_dir, test_dir]:
    os.makedirs(path, exist_ok=True)

# For each class folder
for label in os.listdir(source_dir):
    class_path = os.path.join(source_dir, label)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    images = images[-1000:] 
    random.shuffle(images)

    split_index = int(len(images) * 0.8)
    train_images = images[:split_index]
    test_images = images[split_index:]

    # Create class folders in train and test
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label), exist_ok=True)

    # Copy training images
    for img in tqdm(train_images, desc=f"Train: {label}"):
        src = os.path.join(class_path, img)
        dst = os.path.join(train_dir, label, img)
        shutil.copy(src, dst)

    # Copy test images
    for img in tqdm(test_images, desc=f"Test: {label}"):
        src = os.path.join(class_path, img)
        dst = os.path.join(test_dir, label, img)
        shutil.copy(src, dst)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image shape (we'll resize all images to this)
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 32

#  Define train and test directories
train_dir = "/content/wafer_split/train"
test_dir = "/content/wafer_split/test"

#  Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#  Create Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # For multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Number of classes from train_generator
num_classes = len(train_generator.class_indices)

# ✅ Define CNN model
model = Sequential([
    # Convolution Layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolution Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten and Dense layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')  # Final layer
])


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


model.summary()

In [ ]:
EPOCHS = 20  # You can increase later

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator
)


In [ ]:
import matplotlib.pyplot as plt

# Accuracy Plot
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Loss Plot
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
test_loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
# ✅ Evaluate on training data
train_loss, train_accuracy = model.evaluate(train_generator)
print(f"Train Loss: {train_loss:.4f}")
print(f"Train Accuracy: {train_accuracy:.4f}")


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# True labels
y_true = test_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
labels = list(test_generator.class_indices.keys())

# Plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Optional: Full classification report
print(classification_report(y_true, y_pred, target_names=labels))


In [ ]:
import random
from tensorflow.keras.preprocessing import image

# Pick a random image path from test folder
test_class = random.choice(os.listdir(test_dir))
test_class_path = os.path.join(test_dir, test_class)
test_image_file = random.choice(os.listdir(test_class_path))
test_image_path = os.path.join(test_class_path, test_image_file)

# Load and preprocess the image
img = image.load_img(test_image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

# Predict
prediction = model.predict(img_array)
predicted_class = labels[np.argmax(prediction)]

# Show image and prediction
plt.imshow(img)
plt.axis('off')
plt.title(f"Predicted: {predicted_class}")
plt.show()


In [ ]:
model.save("wafer_defect_model.h5")


In [ ]:
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("wafer_defect_model.h5")
